# 特殊な加工・可視化を行なう10本 Knock

## 大容量CSV data を扱ってみよう
容量無制限に処理できるものではない為、Memory容量との兼ね合いなどを考慮しつつ工夫する必要がでてくる、

※時系列Data を大容量Data と仮定して使用し演習を実施。Data が小さくても Code の書き方と結果を確認し Image を掴む。

In [1]:
# File を読み込む
import pandas as pd
df = pd.read_csv('data/chapter8/person_count_out_0001_2021011509.csv')
df

,id,place,receive_time,sensor_num,in1,out1,state1,in2,out2,state2
0,0,1,2021-01-15 09:00:00.144,2,508,73,0,73,508,0
1,1,1,2021-01-15 09:00:01.146,2,508,73,0,73,508,0
2,2,1,2021-01-15 09:00:02.161,2,508,73,0,73,508,0
3,3,1,2021-01-15 09:00:03.176,2,508,73,0,73,508,0
4,4,1,2021-01-15 09:00:04.192,2,508,73,0,73,508,0
...,...,...,...,...,...,...,...,...,...,...
3535,3535,1,2021-01-15 09:59:55.054,2,782,156,0,156,782,0
3536,3536,1,2021-01-15 09:59:56.07,2,782,156,0,156,782,0
3537,3537,1,2021-01-15 09:59:57.085,2,782,156,0,156,782,0
3538,3538,1,2021-01-15 09:59:58.101,2,782,156,0,156,782,0


仮に Data数が 100万~1000万以上の Data になると pandas.read_csv() で一括で読み込もうとすると OOM Error で Program が落ちると想定される。
※ OOM Error: Memory不足による Error

In [2]:
# OOM Error を回避する為に引数で桁数を指定して読み込む
for df in pd.read_csv('data/chapter8/person_count_out_0001_2021011509.csv', chunksize=512):
    print(df.shape)

(512, 10)
(512, 10)
(512, 10)
(512, 10)
(512, 10)
(512, 10)
(468, 10)


`chunksize=` 明示的に指定すると指定した行数ごとに CSV File を読み込む

In [3]:
# 読み込んだ Dataに対して何らかの処理を行なった上で、別File に保存してみる
i = 0
for df in pd.read_csv('data/chapter8/person_count_out_0001_2021011509.csv', chunksize=64):  # Chunksize を指定して読み込み
    df['processed_per_chunk'] = True  # column を追加
    df.to_csv('data/chapter8/processed_big_data.csv', mode='a', index_label=False, header=i == 0)  # 新しい File に保存していく
    # media='a' と指定することで同じ File に対して追記して保存する。（※'a': append の頭文字）
    # header=i == 0 とすることで i == 0 の時だけ header ありで出力するようにしている

    i += 1

In [4]:
# 結果の確認
from IPython.display import display
df = pd.read_csv('data/chapter8/processed_big_data.csv')
display(df)

,id,place,receive_time,sensor_num,in1,out1,state1,in2,out2,state2,processed_per_chunk
0,0,1,2021-01-15 09:00:00.144,2,508,73,0,73,508,0,True
1,1,1,2021-01-15 09:00:01.146,2,508,73,0,73,508,0,True
2,2,1,2021-01-15 09:00:02.161,2,508,73,0,73,508,0,True
3,3,1,2021-01-15 09:00:03.176,2,508,73,0,73,508,0,True
4,4,1,2021-01-15 09:00:04.192,2,508,73,0,73,508,0,True
...,...,...,...,...,...,...,...,...,...,...,...
3535,3535,1,2021-01-15 09:59:55.054,2,782,156,0,156,782,0,True
3536,3536,1,2021-01-15 09:59:56.07,2,782,156,0,156,782,0,True
3537,3537,1,2021-01-15 09:59:57.085,2,782,156,0,156,782,0,True
3538,3538,1,2021-01-15 09:59:58.101,2,782,156,0,156,782,0,True


## JSON 形式の File を扱ってみよう

In [5]:
pd.read_json('data/chapter8/column_oriented.json')

,id,value
0,1,1
1,2,10
2,3,100


In [6]:
# Pandas で読み込む前の状態を確認
!type data\chapter8\column_oriented.json

{"id":{"0":1,"1":2,"2":3},"value":{"0":1,"1":10,"2":100}}


列指向の JSON File という

In [7]:
!type data\chapter8\index_oriented.json

{"0":{"id":1,"value":1},"1":{"id":2,"value":10},"2":{"id":3,"value":100}}


Index指向の JSON File である

In [8]:
pd.read_json('data/chapter8/index_oriented.json')

,0,1,2
id,1,2,3
value,1,10,100


縦横が逆になっている

In [9]:
pd.read_json('data/chapter8/index_oriented.json', orient='index')
# 引数 orient='index' を指定することで Index指向のJSON File が正しく読み込める

,id,value
0,1,1
1,2,10
2,3,100


In [10]:
# Table 指向な構造の JSON File（※ RDB の File を JSON File に dump する際に見られる構造）
!type data\chapter8\table_oriented.json

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"id","type":"integer"},{"name":"value","type":"integer"}],"primaryKey":["index"],"pandas_version":"0.20.0"},"data":[{"index":0,"id":1,"value":1},{"index":1,"id":2,"value":10},{"index":2,"id":3,"value":100}]}


In [11]:
# そのまま Pandas で読み込んでみる
pd.read_json('data/chapter8/table_oriented.json')

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

Error になる

In [12]:
pd.read_json('data/chapter8/table_oriented.json', orient='table')
# 引数 orient='table' を指定することで Table指向の JSON File も読み込める

,id,value
0,1,1
1,2,10
2,3,100


JSON 形式には様々な構造が存在するため、読み込む際は注意が必要。
※今回の演習で登場した構造以外にも pandas.read_json() で読み込める構造がある。

## Web からの Data を取得してみよう
Web site に対して Request を http送信した際、Server が Client に結果を送信する Case では、JSON形式が使われる場合が多い。
上記の Case の場合の Data の扱う

In [14]:
# worldtimeapi.org という Site に http request を送信し、東京の時刻等の情報を取得して、Contents を表示する
import requests  # http request を扱う Package として requests を選択
response = requests.get('https://worldtimeapi.org/api/timezone/Asia/Tokyo')
response.content

b'{"abbreviation":"JST","client_ip":"121.119.11.5","datetime":"2022-06-08T15:21:46.893652+09:00","day_of_week":3,"day_of_year":159,"dst":false,"dst_from":null,"dst_offset":0,"dst_until":null,"raw_offset":32400,"timezone":"Asia/Tokyo","unixtime":1654669306,"utc_datetime":"2022-06-08T06:21:46.893652+00:00","utc_offset":"+09:00","week_number":23}'

Response が JSON形式の Data であることを確認。

In [15]:
# JSONのままでは扱いずらいため dict型に変換
result = response.json()  # Response.json() で dict型に変換可能
result

{'abbreviation': 'JST',
 'client_ip': '121.119.11.5',
 'datetime': '2022-06-08T15:21:46.893652+09:00',
 'day_of_week': 3,
 'day_of_year': 159,
 'dst': False,
 'dst_from': None,
 'dst_offset': 0,
 'dst_until': None,
 'raw_offset': 32400,
 'timezone': 'Asia/Tokyo',
 'unixtime': 1654669306,
 'utc_datetime': '2022-06-08T06:21:46.893652+00:00',
 'utc_offset': '+09:00',
 'week_number': 23}

Key と値の関係について視認性が向上した。

In [16]:
# Pandas.Series に変換してみる
pd.Series(result)  # Pandas.Series() に引数を渡すことで Series に変換

abbreviation                                 JST
client_ip                           121.119.11.5
datetime        2022-06-08T15:21:46.893652+09:00
day_of_week                                    3
day_of_year                                  159
dst                                        False
dst_from                                    None
dst_offset                                     0
dst_until                                   None
raw_offset                                 32400
timezone                              Asia/Tokyo
unixtime                              1654669306
utc_datetime    2022-06-08T06:21:46.893652+00:00
utc_offset                                +09:00
week_number                                   23
dtype: object

In [17]:
# 取得結果を保存しておく
import json  # Json Libray を import

with open('dump/response.json', mode='w') as f:  # with open に出力File 名を記載して mode='w'(書き込みMode)で Open.
    json.dump(result, f)  # Json.dump() で File に内容を書き込み

In [19]:
# 定期的に Request を実行し、結果を１つの File に追記する
import time  # time Library を import

for _ in range(4):  # renge(4)で４回繰り返し処理
    response = requests.get('https://worldtimeapi.org/api/timezone/Asia/Tokyo')
    with open('dump/response.txt', mode='a') as f:  # mode='a' で追記Mode で File を Open
        res = response.json()  # 変数に response を Response.json() で dict型で格納
        f.write(f"{json.dumps(res)}\n")  # File に変数に格納した Data を出力していく
    time.sleep(1)  # １秒置く

In [20]:
# 出力した Data を確認
!type dump\response.txt

{"abbreviation": "JST", "client_ip": "121.119.11.5", "datetime": "2022-06-08T15:38:48.264271+09:00", "day_of_week": 3, "day_of_year": 159, "dst": false, "dst_from": null, "dst_offset": 0, "dst_until": null, "raw_offset": 32400, "timezone": "Asia/Tokyo", "unixtime": 1654670328, "utc_datetime": "2022-06-08T06:38:48.264271+00:00", "utc_offset": "+09:00", "week_number": 23}
{"abbreviation": "JST", "client_ip": "121.119.11.5", "datetime": "2022-06-08T15:38:49.319286+09:00", "day_of_week": 3, "day_of_year": 159, "dst": false, "dst_from": null, "dst_offset": 0, "dst_until": null, "raw_offset": 32400, "timezone": "Asia/Tokyo", "unixtime": 1654670329, "utc_datetime": "2022-06-08T06:38:49.319286+00:00", "utc_offset": "+09:00", "week_number": 23}
{"abbreviation": "JST", "client_ip": "121.119.11.5", "datetime": "2022-06-08T15:38:50.398317+09:00", "day_of_week": 3, "day_of_year": 159, "dst": false, "dst_from": null, "dst_offset": 0, "dst_until": null, "raw_offset": 32400, "timezone": "Asia/Tokyo", 